In [196]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
from datetime import timedelta
import re

In [197]:

lac_2022_df = pd.read_csv('data/lac_2022_pbp.csv')
lac_2022_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback."
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...


In [198]:
roster_df = pd.read_csv('data/lac_2022_roster.csv')
roster_list=roster_df['roster'].tolist()
print(roster_list)
#roster_df.head()

['Nasir Adderley', 'Keenan Allen', 'Jeremiah Attaochu', 'Zack Bailey', 'Michael Bandy', 'Taylor Bertolet', 'Joey Bosa', 'Bryce Callahan', 'DeAndre Carter', 'Will Clapp', 'Christian Covington', 'Chase Daniel', 'Michael Davis', 'Tyeler Davison', 'Cameron Dicker', 'Austin Ekeler', 'Gerald Everett', 'Breiden Fehoko', 'Matt Feiler', 'Morgan Fox', 'Joe Gaziano', 'Alohi Gilman', 'Jalen Guyton', 'Kemon Hall', 'Josh Harris', 'Justin Herbert', 'Christopher Hinton', 'Dustin Hopkins', 'Zander Horvath', 'J.C. Jackson', 'Brenden Jaimes', 'Derwin James', 'Austin Johnson', 'Zion Johnson', 'Sebastian Joseph-Day', 'Joshua Kelley', 'Raheem Layne', 'Deane Leonard', 'Corey Linsley', 'Khalil Mack', "Tre' McKitty", 'Sony Michel', 'David Moa', 'Jason Moore', 'Kenneth Murray', 'Nick Niemann', 'Storm Norton', 'Amen Ogbongbemiga', 'Otito Ogbonnia', 'Josh Palmer', 'Donald Parham', 'Trey Pipkins', 'Troy Reeder', 'Richard Rodgers', 'Larry Rountree', 'Chris Rumph', 'Jamaree Salyer', 'Asante Samuel Jr.', 'Foster Sare

In [199]:
def possession_check(sentence, roster_list,team_1_abrv,):
    # Split the sentence into words
    words = sentence.split()
    # each detail begins with the name of a player on the 
    name = ' '.join(words[:2])
    # Check if the name is on thhe roster list, if not then it is the other
    if name in roster_list:
        return team_1_abrv
    else:
        return #"OPP"

# Define your list of words to check against
# Apply the function to create a new column called "possession_team"






In [200]:
lac_2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   week       3063 non-null   int64  
 1   Quarter    3062 non-null   object 
 2   Time       3062 non-null   object 
 3   Down       2660 non-null   float64
 4   ToGo       2660 non-null   float64
 5   Location   2941 non-null   object 
 6   OPP        3063 non-null   object 
 7   OPP Score  2940 non-null   float64
 8   LAC Score  2940 non-null   float64
 9   Detail     3063 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 239.4+ KB


In [201]:
lac_2022_df['possession_team'] = lac_2022_df['Detail'].apply(lambda x: possession_check(x, roster_list, "LAC"))
#lac_2022_df_clean = lac_2022_df_clean[lac_2022_df_clean['possession_team'] == 'CLE']
#lac_2022_df = lac_2022_df[lac_2022_df['Down'] != 4]
lac_off_df = lac_2022_df.copy()
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC


In [202]:
lac_off_df = lac_off_df[(lac_off_df['Quarter'] != "OT")]
lac_off_df = lac_off_df.dropna()
lac_off_df = lac_off_df.astype({'Quarter': 'int8', 'Down': 'int8','ToGo': 'int8','Down': 'int8','Location':'string','OPP':'string','OPP Score':'int8','Detail':'string'})
#'Time':'string'
lac_off_df_clean=lac_off_df.copy()
lac_off_df_clean.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC


In [203]:
# Set initial value for the first row
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_1_2'] = lac_off_df.at[lac_off_df.index[0], 'ToGo']
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_2_3'] = 0

# Iterate through the DataFrame to calculate 'yardage_diff_1_2'
for index, row in lac_off_df.iterrows():
    if row['Down'] == 1:  # If Down is 1, set 'yardage_diff_1_2' to "yards to go"
        lac_off_df.at[index, 'yardage_diff_1_2'] = row['ToGo']
    elif row['Down'] == 2:
        if index - 1 in lac_off_df.index:
            lac_off_df.at[index, 'yardage_diff_1_2'] = abs(row['ToGo'] - lac_off_df.at[index - 1, 'ToGo'])
        else:
            lac_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 3:
        lac_off_df.at[index, 'yardage_diff_1_2'] = 0

# Set the value of 'yardage_diff_2_3' for all rows (since it remains constant)
lac_off_df['yardage_diff_2_3'] = 0

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0


In [204]:
for index, row in lac_off_df.iterrows():
    if index > 2 and row['Down'] == 3:  # Exclude the first two rows and only calculate for Down == 3
        if (index - 1) in lac_off_df.index:  # Check if the previous index exists
            lac_off_df.at[index, 'yardage_diff_2_3'] = lac_off_df.at[index - 1, 'ToGo'] - row['ToGo']
        else:
            lac_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 if the previous index doesn't exist
    else:
        lac_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 for other rows or when Down != 3
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0


In [205]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bintercepted\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'intercepted'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'intercepted'] = 0
        
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,intercepted
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0,0.0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0,0.0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1,0.0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0,0.0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0,0.0


In [206]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfumbles\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'fumbled'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'fumbled'] = 0
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumbled
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0,0.0,0.0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0,0.0,0.0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1,0.0,0.0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0,0.0,0.0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0,0.0,0.0


In [207]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bpunts\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'punted'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'punted'] = 0
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumbled,punted
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0,0.0,0.0,0.0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0,0.0,0.0,0.0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1,0.0,0.0,0.0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0,0.0,0.0,0.0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0,0.0,0.0,0.0


In [208]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfield goal\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'field goal att'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'field goal att'] = 0
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumbled,punted,field goal att
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0,0.0,0.0,0.0,0.0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0,0.0,0.0,0.0,0.0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1,0.0,0.0,0.0,0.0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0,0.0,0.0,0.0,0.0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0,0.0,0.0,0.0,0.0


In [209]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bsacked\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'sacked'] = 1  # Set 'sacked' to 1 if 'sacked' is found
        # Extract sack yards using regex
        sack_yards_match = re.search(r'(-?\d+)\s+yards', play_detail)  # Match negative or positive numbers
        if sack_yards_match:
            sack_yards = int(sack_yards_match.group(1))  # Extract the sack yards value
            lac_off_df.at[index, 'sack_yards'] = sack_yards  # Assign sack yards to the corresponding row
        else:
            lac_off_df.at[index, 'sack_yards'] = 0  # If no sack yards found, set it to 0
    else:
        lac_off_df.at[index, 'sacked'] = 0
        lac_off_df.at[index, 'sack_yards'] = 0
        
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumbled,punted,field goal att,sacked,sack_yards
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,10.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,LAC,8.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,10.0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,LAC,2.0,0,0.0,0.0,0.0,0.0,0.0,0.0


for index, row in lac_2022_df_clean.iterrows():
    if index == 0:  
        lac_2022_df_clean.at[index, 'time_between_plays'] = 0  # or NaN
    elif row['Down'] > 1:
        if index - 1 in lac_2022_df_clean.index:
            lac_2022_df_clean.at[index, 'yardage_diff'] = abs(row['ToGo'] - lac_2022_df_clean.at[index - 1, 'ToGo'])
        else:
            lac_2022_df_clean.at[index, 'yardage_diff'] = 0  # or NaN
    elif row['Down'] == 1:
        lac_2022_df_clean.at[index, 'yardage_diff'] = row['ToGo']
lac_2022_df_clean.head()

In [210]:
def update_pass_columns(row):
    global previous_pass_tally  # Make previous_pass_tally accessible and modifiable within the function
    detail = row['Detail']
    if "pass incomplete" in detail:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 1
        row['pass_play'] = 1
        row['pass_length'] = 0
    elif "pass complete" in detail:
        yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
        if yards_match:
            yards = int(yards_match.group(1))
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = yards
            previous_pass_tally += yards  # Update pass yards tally
        else:
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = 0
    else:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 0
        row['pass_play'] = 0
        row['pass_length'] = 0
    
    row['pass_yards_tally'] = previous_pass_tally  # Update pass yards tally
    return row

# Initialize cumulative passing yards
previous_pass_tally = 0

# Apply the function to each row to update pass columns
lac_off_df = lac_off_df.apply(update_pass_columns, axis=1)


In [211]:
def update_rush_columns(row):
    detail = row['Detail']
    match_words = ['middle for', 'tackle for', 'guard for', 'end for']

    for word in match_words:
        if word in detail:
            yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
            if yards_match:
                row['rush_play'] = 1
                row['rush_length'] = int(yards_match.group(1))
            else:
                row['rush_play'] = 0
                row['rush_length'] = 0
            return row
    
    row['rush_play'] = 0
    row['rush_length'] = 0
    return row

def update_rush_yards_tally(row, previous_tally):
    if row['rush_play'] == 1:
        row['rush_yards_tally'] = previous_tally + row['rush_length']
    else:
        row['rush_yards_tally'] = previous_tally
    return row

# Apply the function to each row to update rush columns
# Apply the function to each row to update rush columns
lac_off_df = lac_off_df.apply(lambda row: update_rush_columns(row), axis=1)


# Calculate cumulative rushing yards
cumulative_rush_yards = 0
lac_off_df['rush_yards_tally'] = 0
for index, row in lac_off_df.iterrows():
    lac_off_df.loc[index] = update_rush_yards_tally(row, cumulative_rush_yards)
    cumulative_rush_yards = lac_off_df.loc[index, 'rush_yards_tally']

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,sacked,sack_yards,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0,0,0,0,0,1,8,8
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,1,0,1,0,0,0,0,8
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0,0,0,0,0,1,2,10
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0,0,0,0,0,1,2,12
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,0.0,1,0,1,9,9,0,0,12


In [212]:
def yards_to_promised_land(location):
    match = re.search(r'CLE\s+(\d+)', location)
    if match:
        yards = int(match.group(1))
        return 50 + yards
    else:
        return int(location.split()[1])


# Apply the function to the 'Detail' column and create a new column 'yards_to_endzone'
lac_off_df['yards_to_endzone'] = lac_off_df['Location'].apply(yards_to_promised_land)
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,sack_yards,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0,0,0,0,0,1,8,8,25
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,1,0,1,0,0,0,0,8,33
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0,0,0,0,0,1,2,10,34
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0,0,0,0,0,1,2,12,36
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,1,0,1,9,9,0,0,12,38


create new column "conversion_successful" if  while on third down the resultmof the play is a rush length or pass length is greater than 'yards to go', then put 1 in the conversion seuccesfful 

In [213]:
def update_1d_conversion_successful(row):
    if row['Down'] == 1:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['1D_conv_good'] = lac_off_df.apply(update_1d_conversion_successful, axis=1)
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_complete,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0,0,0,0,0,1,8,8,25,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,1,0,1,0,0,0,0,8,33,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0,0,0,0,0,1,2,10,34,0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0,0,0,0,0,1,2,12,36,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,1,0,1,9,9,0,0,12,38,0


In [214]:
def update_2d_conversion_successful(row):
    if row['Down'] == 2:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['2D_conv_good'] = lac_off_df.apply(update_2d_conversion_successful, axis=1)
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_incomplete,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good,2D_conv_good
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0,0,0,0,1,8,8,25,0,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,0,1,0,0,0,0,8,33,0,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0,0,0,0,1,2,10,34,0,0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0,0,0,0,1,2,12,36,0,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,0,1,9,9,0,0,12,38,0,1


In [215]:
def update_3d_conversion_successful(row):
    if row['Down'] == 3:  # Check if it's third down
        if row['rush_play'] == 1:  # If it's a rush play
            if row['rush_length'] >= row['ToGo']:  # Check if rushing yards are greater than or equal to yards to go
                return 1
        elif row['pass_play'] == 1:  # If it's a pass play
            if row['pass_length'] >= row['ToGo']:  # Check if pass length is greater than or equal to yards to go
                return 1
    return 0
# Apply the function to each row to update the conversion_successful column
lac_off_df['3D_conv_good'] = lac_off_df.apply(update_3d_conversion_successful, axis=1)
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_play,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0,0,0,1,8,8,25,0,0,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,1,0,0,0,0,8,33,0,0,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0,0,0,1,2,10,34,0,0,1
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0,0,0,1,2,12,36,0,0,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,1,9,9,0,0,12,38,0,1,0


In [216]:
def update_4d_conversion_successful(row):
    if row['Down'] == 4:  # Check if it's third down
        if row['rush_play'] == 1:  # If it's a rush play
            if row['rush_length'] >= row['ToGo']:  # Check if rushing yards are greater than or equal to yards to go
                return 1
        elif row['pass_play'] == 1:  # If it's a pass play
            if row['pass_length'] >= row['ToGo']:  # Check if pass length is greater than or equal to yards to go
                return 1
    return 0
# Apply the function to each row to update the conversion_successful column
lac_off_df['4D_conv_good'] = lac_off_df.apply(update_4d_conversion_successful, axis=1)
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_length,pass_yards_tally,rush_play,rush_length,rush_yards_tally,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good,4D_conv_good
1,1,1,15:00,1,10,LAC 25,LVR,0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0,0,1,8,8,25,0,0,0,0
2,1,1,14:45,2,2,LAC 33,LVR,0,0.0,Justin Herbert pass complete short right to Ke...,...,0,0,0,0,8,33,0,0,0,0
3,1,1,14:07,3,1,LAC 34,LVR,0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0,0,1,2,10,34,0,0,1,0
4,1,1,13:24,1,10,LAC 36,LVR,0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0,0,1,2,12,36,0,0,0,0
5,1,1,12:50,2,8,LAC 38,LVR,0,0.0,Justin Herbert pass complete short left to Jos...,...,9,9,0,0,12,38,0,1,0,0
